# Assistant-04<br/>Function calling

## Trading Assistant with email functionality

## Credits

- https://mer.vin/2023/11/openai-assistants-api-function-calling/

## Email functionality

- Implemented using Logic Apps with an Http POST trigger

### Get an OpenAI client and an Assistant

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import common
import time
import requests
import oaihelper as framework
import yfinance as yf


## Get an OpenAI client
client = framework.get_openai_client()

### Get the latest stock price by ticker symbol using Yahoo Finance

In [2]:
def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    price = stock.history(period="1d")['Close'].iloc[-1]    
    return price


### Use a Logic App HTTP POST trigger to email using Outlook

In [3]:
def send_post_request(json_payload):
    headers = {'Content-Type': 'application/json'}
    response = requests.post(framework.email_URI, json=json_payload, headers=headers)
    if response.status_code == 202:
        print("Email sent to: " + json_payload['to'])  

### Prepare the tool for function calling

In [4]:
tools_list = [{
    "type": "function",
    "function": {

        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
    }
},
{
    "type": "function",
    "function": {

        "name": "send_email",
        "description": "sends an email to a recipient(s).",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {
                    "type": "string",
                    "description": "The email(s) the email should be sent to."
                },
                "content": {
                    "type": "string",
                    "description": "The content of the email."
                }
            },
            "required": ["to", "content"]
        }
    }
}]

### Create the assistant

In [5]:
framework.clear_shelves()

## Create an assistant
stock_assistant = framework.create_assistant(client,
                                                  name="fta-Securities Trading Assistant",
                                                  instructions="You are a personal securities trading assistant. Please be polite, professional, helpful, and friendly.", 
                                                  tools=tools_list, 
                                                  model=common.gpt_deployment_name)

Added assistant:  asst_PVeWX15ci4dQJaDNrEEMAlx7 1


### Create a thread, put a message on the thread, and view the initial status

In [6]:
# Step 2: Create a Thread
thread = client.beta.threads.create()

article = """Dow Jones futures will open Sunday evening, along with S&P 500 futures and Nasdaq futures.

The stock market rally had a strong rebound this past week after the major indexes and many leading stocks retreated to start 2024 but soon found support at key levels. A large number of stocks flashed buy signals, including a powerful breakout from Nvidia (NVDA).

It's been a good time to add exposure.

Nvidia stock is now extended, but fellow AI leader Microsoft (MSFT) is just above a buy point after solid weekly gains, on the cusp of surpassing the market cap of fellow Dow giant Apple (AAPL). Novo Nordisk (NVO) remains in a buy zone.

MercadoLibre (MELI) and Tradeweb Markets (TW) flashed entries intraday.

On the downside, Tesla (TSLA) had an ugly week, extending a recent sell-off and breaking key support levels amid a flurry of headlines.

Nvidia and NVO stock are on IBD Leaderboard. MELI stock, Nvidia and Microsoft are on SwingTrader. MSFT stock is on IBD Long-Term Leaders. Nvidia stock, MercadoLibre, Novo Nordisk and Microsoft are on the IBD 50. Microsoft, Nvidia and MercadoLibre stock are on the IBD Big Cap 20."""

# Step 3: Add a Message to a Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="For the following article: \n\nArticle: """"""" + article + "\n""""""" +
    "\n\nEmail a summary of the article in simple HTML format, include today's date, include a list of the closing prices of the stocks mentioned in the article to: alemor@microsoft.com." +
    "Use only the provide text."
)

# Step 4: Run the Assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=stock_assistant.id,
    instructions="Today's date and time is: " + framework.get_localized_datetime() + ".\n\n"
)

#print(run.model_dump_json(indent=4))

### Process the message, function calling leveraging the latest stock prices, print the results and dispose of the objects

In [7]:
while True:
    # Wait for 1 seconds
    time.sleep(1)

    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    #print(run_status.model_dump_json(indent=4))

    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )

        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")

        ## Cleanup
        print(client.beta.threads.delete(thread.id))
        print(client.beta.assistants.delete(stock_assistant.id))

        break
    elif run_status.status == 'requires_action':
        print("Function Calling")
        required_actions = run_status.required_action.submit_tool_outputs.model_dump()
        print(required_actions)
        tool_outputs = []
        import json
        for action in required_actions["tool_calls"]:
            func_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])
            
            if func_name == "get_stock_price":
                output = get_stock_price(symbol=arguments['symbol'])
                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": output
                })
            elif func_name == "send_email":
                print("Sending email...")
                email_to = arguments['to']
                email_content = arguments['content']
                send_post_request({'to': email_to,'content':email_content})
                
                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": "Email sent"
                })
            else:
                raise ValueError(f"Unknown function: {func_name}")
            
        print("Submitting outputs back to the Assistant...")
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    else:
        print("Waiting for the Assistant to process...")
        time.sleep(1)

Waiting for the Assistant to process...
Waiting for the Assistant to process...
Waiting for the Assistant to process...
Function Calling
{'tool_calls': [{'id': 'call_NZZmqq3vy2kNLD0nxXVj56PG', 'function': {'arguments': '{"symbol": "NVDA"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_L7ZjL7l44Ss9xw5O0fSAz3he', 'function': {'arguments': '{"symbol": "MSFT"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_dn8TGQOpr1zG7uRdZaOrs6xK', 'function': {'arguments': '{"symbol": "AAPL"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_08ufucLncSGySFXDEGtvzrOP', 'function': {'arguments': '{"symbol": "NVO"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_fXlE0wHuLMrZaXngHMP0ynkY', 'function': {'arguments': '{"symbol": "MELI"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_TaTo5BWEPCpEdhezaEwTYhzC', 'function': {'arguments': '{"symbol": "TW"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_kjeh3poyrVhhxsc